In [80]:
import pandas as pd

In this notebook we will prepare the data.....

### Import data

In [81]:
games = pd.read_csv('games.csv')
games_stats = pd.read_csv('games_stats.csv')

games.rename({'id':'game_id'},axis=1,inplace=True)

### Give a name

In [82]:
# create target varible - home win = 1 , away win = 0

def game_result(h_score, a_score):
    if h_score > a_score:
        return 1
    else:
        return 0

games['result'] = games.apply(lambda x: game_result(x['H_score'], x['A_score']), axis=1)

In [83]:
#join dates info to games_stats dataframe

games_stats[['date','Home_team_id','game_h_team_result']] = \
games_stats.merge(games, on='game_id', how='left' )[['date','H_team_id','result']]

#add result to game_stats 

def game_true_result(team_id, h_team_id, result):
    if team_id == h_team_id:
        return result
    elif result == 0:
        return 1
    else: return 0

games_stats['game_true_result'] = games_stats.apply(lambda x: game_true_result(x['team_id'],x['Home_team_id'],x['game_h_team_result']),axis=1)
games_stats.drop(['Home_team_id','game_h_team_result'],axis = 1, inplace = True)

In [84]:
#win percentage 

games_stats.sort_values(by = ['team_id','date'], ascending = True, inplace = True)
team_stats_groupby = games_stats.groupby(['team_id'])

games_stats['game_number'] = team_stats_groupby.cumcount()
games_stats['wins_before'] = team_stats_groupby['game_true_result'].apply(lambda x : x.shift().cumsum())
games_stats['w_per_before'] = games_stats['wins_before']/games_stats['game_number']

games_stats['w_per_last10games'] = team_stats_groupby['game_true_result'] \
                    .rolling(10, closed='left').sum() \
                    .reset_index(drop=True, level=0)
games_stats['w_per_last5games'] = team_stats_groupby['game_true_result'] \
                    .rolling(5, closed='left').sum() \
                    .reset_index(drop=True, level=0) 




In [85]:
# team stats rolling avg 10 games 


columns_to_aggregate = ['fastBreakPoints', 'pointsInPaint', 'biggestLead',
       'secondChancePoints', 'pointsOffTurnovers', 'longestRun', 'points',
       'fgm', 'fga', 'fgp', 'ftm', 'fta', 'ftp', 'tpm', 'tpa', 'tpp', 'offReb',
       'defReb', 'totReb', 'assists', 'pFouls', 'steals', 'turnovers',
       'blocks', 'plusMinus']

columns_aggregated = ['mean_' + col for col in columns_to_aggregate]

games_stats[columns_aggregated] = \
    team_stats_groupby[columns_to_aggregate].rolling(10,  closed='left').mean().reset_index(drop=True, level=0)

relevant_columns = ['game_id','team_id'] + columns_aggregated +['w_per_before','w_per_last10games','w_per_last5games']




In [86]:
# Home team data merge
full_data = games.merge(right=games_stats[relevant_columns].add_prefix('H_'), left_on=['game_id','H_team_id'],right_on=['H_game_id','H_team_id'])
# Away team data merge
full_data = full_data.merge(right=games_stats[relevant_columns].add_prefix('A_'), left_on=['game_id','A_team_id'],right_on=['A_game_id','A_team_id'])

full_data

,game_id,season,date,arena,H_team,H_team_id,A_team,A_team_id,H_score,A_score,...,A_mean_totReb,A_mean_assists,A_mean_pFouls,A_mean_steals,A_mean_turnovers,A_mean_blocks,A_mean_plusMinus,A_w_per_before,A_w_per_last10games,A_w_per_last5games
0,8787,2021,2021-10-19T23:30:00.000Z,Fiserv Forum,Milwaukee Bucks,21,Brooklyn Nets,4,127,104,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.600000,NaN,3.0
1,8788,2021,2021-10-20T02:00:00.000Z,STAPLES Center,Los Angeles Lakers,17,Golden State Warriors,11,114,121,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.714286,NaN,5.0
2,8789,2021,2021-10-20T23:00:00.000Z,Spectrum Center,Charlotte Hornets,5,Indiana Pacers,15,123,122,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.600000,NaN,3.0
3,8790,2021,2021-10-20T23:00:00.000Z,Little Caesars Arena,Detroit Pistons,10,Chicago Bulls,6,88,94,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.833333,NaN,5.0
4,8791,2021,2021-10-20T23:30:00.000Z,Madison Square Garden,New York Knicks,24,Boston Celtics,2,138,134,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.400000,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1330,10913,2021,2022-04-16T02:00:00.000Z,Crypto.com Arena,LA Clippers,16,New Orleans Pelicans,23,101,105,...,43.1,24.8,17.7,9.4,12.1,4.0,0.7,0.449438,6.0,3.0
1331,10914,2021,2022-04-17T17:00:00.000Z,FTX Arena,Miami Heat,20,Atlanta Hawks,1,115,91,...,45.6,25.4,19.9,7.7,10.1,3.4,11.0,0.528090,8.0,4.0
1332,10915,2021,2022-04-19T23:30:00.000Z,FTX Arena,Miami Heat,20,Atlanta Hawks,1,115,105,...,45.4,23.8,19.9,7.4,10.6,3.2,7.7,0.522222,7.0,3.0
1333,10918,2021,2022-04-18T01:00:00.000Z,Footprint Center,Phoenix Suns,28,New Orleans Pelicans,23,110,99,...,44.3,24.5,18.4,9.2,12.1,3.2,1.5,0.455556,7.0,3.0


In [88]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1335 entries, 0 to 1334
Data columns (total 75 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   game_id                    1335 non-null   int64  
 1   season                     1335 non-null   int64  
 2   date                       1335 non-null   object 
 3   arena                      1324 non-null   object 
 4   H_team                     1335 non-null   object 
 5   H_team_id                  1335 non-null   int64  
 6   A_team                     1335 non-null   object 
 7   A_team_id                  1335 non-null   int64  
 8   H_score                    1335 non-null   int64  
 9   A_score                    1335 non-null   int64  
 10  ref1                       1335 non-null   object 
 11  ref2                       1335 non-null   object 
 12  ref3                       1335 non-null   object 
 13  ties                       927 non-null    float

In [91]:
full_data.to_csv('model_data.csv',index=False)